In [1]:
from skimage import data, io, filters
import numpy as np
from numpy import array
from numpy.random import randint
import pandas as pd
from PIL import Image
import os
import sys
from tqdm import tqdm_notebook

from keras.applications.vgg19 import VGG19
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

from upscaler.data import load_images_from_dir, crop_images, select_random_rows, convert_image_series_to_array, downscale_images, crop_images_cgc
from upscaler.data import select_random_rows
from upscaler.model import make_upscaler_attention, make_discriminator_simple_512, wasserstein_loss, VGG_LOSS, make_and_compile_gan

Using TensorFlow backend.


In [2]:
images_fullhd = load_images_from_dir(
    '../images/ukiyo-e_fullhd/',
    '.jpg',
    limit = 4000,
    prog_func = tqdm_notebook
)
images_fullhd = downscale_images(
    images_fullhd,
    prog_func = tqdm_notebook,
    downscale_ratio = 4
)
images_fullhd = images_fullhd.rename(columns={'image_hr': 'fullhd', 'downscaled' : 'scaled'})

images_1gen = load_images_from_dir(
    '../images/ukiyo-e_1gen/',
    '.jpg',
    limit = 4000,
    prog_func = tqdm_notebook
)
images_1gen = images_1gen.rename(columns={'image_hr': 'gen1'}).drop(columns='image_size')

images_2gen = load_images_from_dir(
    '../images/ukiyo-e_2gen/',
    '.jpg',
    limit = 4000,
    prog_func = tqdm_notebook
)
images_2gen = images_2gen.rename(columns={'image_hr': 'gen2'}).drop(columns='image_size')

images = images_fullhd.join(images_1gen.set_index('filename'), on = 'filename', how = 'inner').join(images_2gen.set_index('filename'), on = 'filename', how = 'inner')

images = crop_images_cgc(images, target_shape = (512,512), seed = 42, downscale_ratio = 4)

In [3]:
def make_and_compile_gan(
    generator,
    discriminator,
    input_shape,
    output_shape,
    content_loss,
    content_loss_weight,
    discriminator_loss,
    discriminator_loss_weight,
    optimizer=Adam()
):
    
    generator_input_layer = Input(shape=input_shape)
    generator_layer = generator(generator_input_layer)
    generator_training_model = Model(inputs=generator_input_layer, outputs=generator_layer)
    generator_training_model.compile(loss=content_loss, optimizer=optimizer)
    
    discriminator.trainable = True
    discriminator_input_layer = Input(shape=output_shape)
    discriminator_layer = discriminator(discriminator_input_layer)
    discriminator_training_model = Model(inputs=discriminator_input_layer, outputs=discriminator_layer)
    discriminator_training_model.compile(loss=discriminator_loss, optimizer=optimizer)
    
    discriminator.trainable = False
    gan_input_layer = Input(shape=input_shape)
    gan_generator_layer = generator(gan_input_layer)
    gan_discriminator_layer = discriminator(gan_generator_layer)
    gan_training_model = Model(inputs=gan_input_layer, outputs=[gan_generator_layer, gan_discriminator_layer])
    gan_training_model.compile(
        loss=[content_loss, discriminator_loss],
        loss_weights=[content_loss_weight, discriminator_loss_weight],
        optimizer=optimizer
    )
    
    return generator_training_model, discriminator_training_model, gan_training_model

In [3]:
input_shape = (128,128,3)
output_shape = (512,512,3)

generator = make_upscaler_attention(output_shape)
generator.name = 'upscaler'
discriminator = make_discriminator_simple_512(output_shape)
discriminator.name = 'discriminator'

vgg_loss = VGG_LOSS(output_shape).loss
optimizer = Adam()

W1127 22:14:43.711073 140528778274624 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1127 22:14:43.714115 140528778274624 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1127 22:14:43.771136 140528778274624 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1127 22:14:43.771991 140528778274624 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1

NameError: name 'make_vgg_discriminator' is not defined

In [ ]:
gen_train, disc_train, gan_train = make_and_compile_gan(
    generator = generator, discriminator = discriminator,
    input_shape = input_shape, output_shape = output_shape,
    content_loss = vgg_loss, content_loss_weight = 1,
    discriminator_loss = wasserstein_loss, discriminator_loss_weight = 1e-3
)

In [346]:
batch_df = select_random_rows(images, 2)

batch_hr = pd.concat([batch_df.cropped_hd, batch_df.cropped_hd, batch_df.cropped_hd], ignore_index=True)
batch_lr = pd.concat([batch_df.cropped_gen1, batch_df.cropped_gen2, batch_df.cropped_scaled], ignore_index=True)

image_batch_hr  = convert_image_series_to_array(batch_hr)
image_batch_lr  = convert_image_series_to_array(batch_lr)
image_batch_gen = gen_train.predict(image_batch_lr)

disc_real_y = np.ones(2 * 3)
disc_fake_y = -disc_real_y

image_batch_disc = np.concatenate((image_batch_hr, image_batch_gen), axis=0)
image_batch_y = np.concatenate((disc_real_y, disc_fake_y), axis=0)

loss_disc = disc_train.train_on_batch(image_batch_disc, image_batch_y)
loss_gan, loss_gan_gen, loss_gan_disc = gan_train.train_on_batch(image_batch_lr, [image_batch_hr,disc_real_y])

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [347]:
loss_disc

-524.1081

In [348]:
loss_gan

-0.6641275

In [349]:
loss_gan_gen

0.069687985

In [350]:
loss_gan_disc

-733.8154

In [301]:
batch_df = select_random_rows(images, 2)

batch_hr = pd.concat([batch_df.cropped_hd, batch_df.cropped_hd, batch_df.cropped_hd], ignore_index=True)
batch_lr = pd.concat([batch_df.cropped_gen1, batch_df.cropped_gen2, batch_df.cropped_scaled], ignore_index=True)

image_batch_hr  = convert_image_series_to_array(batch_hr)
image_batch_lr  = convert_image_series_to_array(batch_lr)
image_batch_gen = gen_train.predict(image_batch_lr)

disc_real_y = np.ones(2 * 3)
disc_fake_y = -disc_real_y

disc_train.train_on_batch(image_batch_gen, disc_fake_y)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


-1331.9447

In [315]:
batch_df = select_random_rows(images, 2)

batch_hr = pd.concat([batch_df.cropped_hd, batch_df.cropped_hd, batch_df.cropped_hd], ignore_index=True)
batch_lr = pd.concat([batch_df.cropped_gen1, batch_df.cropped_gen2, batch_df.cropped_scaled], ignore_index=True)

image_batch_hr  = convert_image_series_to_array(batch_hr)
image_batch_lr  = convert_image_series_to_array(batch_lr)
image_batch_gen = gen_train.predict(image_batch_lr)

disc_real_y = np.ones(2 * 3)
disc_fake_y = -disc_real_y

disc_train.train_on_batch(image_batch_hr, disc_real_y)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


899.6455

In [378]:
batch_df = select_random_rows(images, 2)

batch_hr = pd.concat([batch_df.cropped_hd, batch_df.cropped_hd, batch_df.cropped_hd], ignore_index=True)
batch_lr = pd.concat([batch_df.cropped_gen1, batch_df.cropped_gen2, batch_df.cropped_scaled], ignore_index=True)

image_batch_hr  = convert_image_series_to_array(batch_hr)
image_batch_lr  = convert_image_series_to_array(batch_lr)
image_batch_gen = gen_train.predict(image_batch_lr)

disc_real_y = np.ones(2 * 3)
disc_fake_y = -disc_real_y

image_batch_disc = np.concatenate((image_batch_hr, image_batch_gen), axis=0)
image_batch_y = np.concatenate((disc_real_y, disc_fake_y), axis=0)

gan_train.train_on_batch(image_batch_lr, [image_batch_hr,disc_real_y])

[0.7420183, 0.075797014, 666.22125]

In [396]:
batch_df = select_random_rows(images, 2)

batch_hr = pd.concat([batch_df.cropped_hd, batch_df.cropped_hd, batch_df.cropped_hd], ignore_index=True)
batch_lr = pd.concat([batch_df.cropped_gen1, batch_df.cropped_gen2, batch_df.cropped_scaled], ignore_index=True)

image_batch_hr  = convert_image_series_to_array(batch_hr)
image_batch_lr  = convert_image_series_to_array(batch_lr)
image_batch_gen = gen_train.predict(image_batch_lr)

disc_real_y = np.ones(2 * 3)
disc_fake_y = -disc_real_y

image_batch_disc = np.concatenate((image_batch_hr, image_batch_gen), axis=0)
image_batch_y = np.concatenate((disc_real_y, disc_fake_y), axis=0)

disc_train.predict(image_batch_disc)

array([[-2.0895395e+11],
       [-4.7791428e+10],
       [-2.0895395e+11],
       [-4.7791428e+10],
       [-2.0895395e+11],
       [-4.7791428e+10],
       [ 1.3247290e+08],
       [ 1.3499280e+08],
       [ 1.3545078e+08],
       [ 1.3401422e+08],
       [ 1.3526914e+08],
       [ 1.3252787e+08]], dtype=float32)

In [402]:
batch_df = select_random_rows(images, 2)

batch_hr = pd.concat([batch_df.cropped_hd, batch_df.cropped_hd, batch_df.cropped_hd], ignore_index=True)
batch_lr = pd.concat([batch_df.cropped_gen1, batch_df.cropped_gen2, batch_df.cropped_scaled], ignore_index=True)

image_batch_hr  = convert_image_series_to_array(batch_hr)
image_batch_lr  = convert_image_series_to_array(batch_lr)
image_batch_gen = gen_train.predict(image_batch_lr)

disc_real_y = np.ones(2 * 3)
disc_fake_y = -disc_real_y

image_batch_disc = np.concatenate((image_batch_hr, image_batch_gen), axis=0)
image_batch_y = np.concatenate((disc_real_y, disc_fake_y), axis=0)

gan_train.predict(image_batch_lr)[1].shape

(6, 1)